# EfficientFrontier.ipynb

-------------------------
## 日経２２５の構成銘柄のうち、時価総額の大きい方から２０銘柄を選んで


以下は、直近の「日経225（Nikkei 225）」構成銘柄のうち、時価総額が大きいものの上位 20銘柄（おおよそ上位20 — 情報公開上の順位付けに基づく）です。出典は時価総額ランキングの公表データ。
(kabudragon.com)

📈 日経225：時価総額上位 約20銘柄
|順位|証券コード|銘柄名|
|---|---|---|
|1|7203|トヨタ自動車|
|2|8306|三菱UFJフィナンシャル・グループ (MUFG)|
|3|6758|ソニーグループ|
|4|9984|ソフトバンクグループ|
|5|6501|日立製作所|
|6|8316|三井住友フィナンシャルグループ (SMFG)|
|7|9983|ファーストリテイリング|
|8|8035|東京エレクトロン|
|9|6857|アドバンテスト|
|10|7974|任天堂|
|11|8411|みずほフィナンシャルグループ|
|12|8058|三菱商事|
|13|7011|三菱重工業|
|14|4519|中外製薬|
|15|6861|キーエンス|
|16|8001|伊藤忠商事|
|17|9432|NTT|
|18|8031|三井物産|
|19|6098|リクルートホールディングス|
|20|8766|東京海上ホールディングス|

⚠️ 補足・注意点

この順位は、2025年12月3日時点の「時価総額ランキング（日経225構成銘柄対象）」に基づきます。
(kabudragon.com)

「時価総額が大きい＝流動性・企業規模が大きい」銘柄ですが、必ずしも日経225における「指数への寄与度」が大きいとは限りません。なぜなら、日経225は「株価平均型（price-weighted）」指数であり、**株価そのもの（＝値がさ株）**がより大きな影響を持つためです。
(ウィキペディア)

時価総額ランキングは上下動しやすいため、数週間〜数ヶ月後に見直すと順位が変わっている可能性があります。

--------------------
## 今選んだ２０銘柄の、過去１年間の株価をYahoo Financeから取得、csvに保存して

了解です！
ここでは 「選んだ20銘柄の過去1年間の株価を Yahoo Finance から取得し、CSV に保存する Jupyter Notebook 完成版」 を作成してお渡しします。

yfinance を使うのが最も安定しており、Yahoo のデータをそのまま取得できます。

In [ ]:
# ================================================
# 日経225 時価総額上位20銘柄の株価を
# 過去1年間取得し、CSVファイルに保存する
# ================================================

import yfinance as yf
import pandas as pd
import datetime as dt
import os

# ---- ① 対象20銘柄（先ほど選んだリスト） ----
tickers = [
    "7203.T", "8306.T", "6758.T", "9984.T", "6501.T",
    "8316.T", "9983.T", "8035.T", "6857.T", "7974.T",
    "8411.T", "8058.T", "7011.T", "4519.T", "6861.T",
    "8001.T", "9432.T", "8031.T", "6098.T", "8766.T"
]

# ---- ② 期間：過去1年間 ----
end = dt.date.today()
start = end - dt.timedelta(days=365)

print("取得期間:", start, "〜", end)

# ---- ③ 保存フォルダ ----
save_dir = "nikkei225_top20_prices"
os.makedirs(save_dir, exist_ok=True)

# ---- ④ 各銘柄のデータ取得と CSV 保存 ----
for t in tickers:
    print(f"Downloading {t} ...")
    df = yf.download(t, start=start, end=end, interval="1d", auto_adjust=False)
    
    # データが空でない場合だけ保存
    if df.empty:
        print(f"⚠ No data for {t}, skipping.")
        continue

    # CSV 出力
    path = os.path.join(save_dir, f"{t.replace('.T','')}_1y.csv")
    df.to_csv(path, encoding="utf-8-sig")

print("✅ 完了：全銘柄の CSV を保存しました。")
print("保存フォルダ:", save_dir)


📁 出力される CSV の例

フォルダ：
```
nikkei225_top20_prices/
├── 7203_1y.csv
├── 8306_1y.csv
├── 6758_1y.csv
...
└── 8766_1y.csv
```

✅ 追加できます

必要なら以下も作成できます：

- 20銘柄の 株価をまとめた1つの CSV（横に並べる形式）
- 20銘柄の 日次リターンの計算
- 相関行列・ヒートマップ
- ポートフォリオ最適化（Markowitz）
- PCA（株価の主成分分析）

どれを追加しましょうか？

-------------------------
## 作成した２０銘柄のファイルから終値を取り出し、日次ヒストリーをもとにポートフォリオの最適化を行い、効率的フロンティアを描くプログラムを書いて。ファイルのヘッダー部分をパースし、日付と終値を正しく読み込んで。

いいですね、その流れでポートフォリオ最適化までいきましょう 💹

前提：

既に nikkei225_top20_prices にディレクトリに20銘柄の1年分データがある想定です。

CSV は必ず以下の形式と仮定：

```
Price,Adj Close,Close,High,Low,Open,Volume
Ticker,7203.T,7203.T,7203.T,7203.T,7203.T,7203.T
Date,,,,,,
2024-12-10,2592.2465,2679.5,2700.0,2667.5,2685.5,29297100
...
```

- 最初の3行を正しくパースし、Date と Close だけの DataFrame を生成
- 全銘柄の Time-series を横に結合
- リターン計算
- 最適化（SciPy があれば SLSQP）
- 効率的フロンティアを描画

In [ ]:
# ==========================================================
# Jupyter Notebook 1セルで完結する
# Nikkei225 上位20銘柄 CSV (特殊3行ヘッダー) 対応版
# ==========================================================

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# ----------------------------------------------------------
# 1. 特殊CSVをパースして Date & Close DataFrame を返す関数
# ----------------------------------------------------------
def load_price_csv(path):
    """
    あなたのCSV形式:
    Price,Adj Close,Close,High,Low,Open,Volume
    Ticker,7203.T,...
    Date,,,,,,
    2024-12-10,...
    の構造を正しくパースして
    index=Date の Close 時系列を返す
    """

    # 1行目（列名）
    header1 = pd.read_csv(path, nrows=1, header=None).iloc[0].tolist()

    # Close 列のインデックスを取得
    close_idx = header1.index("Close")

    # 4行目以降（実データ）だけ読み込む
    df = pd.read_csv(path, skiprows=3, header=None)

    # "Date" 列の位置は 0 列目
    df.columns = header1  # "Price", "Adj Close", "Close", ...

    # 日付
    df["Date"] = pd.to_datetime(df["Price"], format="%Y-%m-%d", errors="coerce")

    # Close を取得（数値化）
    df["Close"] = pd.to_numeric(df["Close"], errors="coerce")

    # 不正値を除去
    df = df.dropna(subset=["Date", "Close"])

    # 出力：Date を index にして Close のみ
    out = df[["Date", "Close"]].set_index("Date")

    return out

# ----------------------------------------------------------
# 2. 20銘柄のCSVを読み込んで prices DataFrame を作成
# ----------------------------------------------------------
data_dir = "nikkei225_top20_prices"

prices = pd.DataFrame()

for file in sorted(os.listdir(data_dir)):
    if not file.endswith(".csv"):
        continue

    ticker = file.replace("_1y.csv", "") + ".T"
    path = os.path.join(data_dir, file)

    df = load_price_csv(path)  # ←完全対応関数
    prices[ticker] = df["Close"]

# 欠損値を落とす
prices = prices.dropna()

print("✓ Prices table loaded:", prices.shape)
display(prices.head())

# ----------------------------------------------------------
# 3. リターン計算（年率換算）
# ----------------------------------------------------------
returns = prices.pct_change().dropna()

mu = returns.mean() * 252
cov = returns.cov() * 252

tickers = prices.columns.tolist()
n_assets = len(tickers)

# ----------------------------------------------------------
# 4. ポートフォリオ性能計算
# ----------------------------------------------------------
def port_perf(w, mu, cov):
    r = np.dot(w, mu)
    v = np.sqrt(np.dot(w, np.dot(cov, w)))
    return r, v

# ----------------------------------------------------------
# 5. 最適化（SciPy / fallback to random）
# ----------------------------------------------------------
use_scipy = True
try:
    from scipy.optimize import minimize
except:
    print("⚠ SciPy not found → using random portfolios")
    use_scipy = False

frontier_r = []
frontier_v = []

if use_scipy:
    mu_arr, cov_arr = mu.values, cov.values
    w0 = np.ones(n_assets) / n_assets
    bounds = [(0, 1)] * n_assets
    targets = np.linspace(mu.min(), mu.max(), 50)

    for t in targets:
        def obj(w):
            return np.dot(w, np.dot(cov_arr, w))  # variance

        cons = [
            {"type": "eq", "fun": lambda w: w.sum() - 1},
            {"type": "eq", "fun": lambda w: np.dot(w, mu_arr) - t},
        ]

        res = minimize(obj, w0, method="SLSQP", bounds=bounds, constraints=cons)

        if res.success:
            r, v = port_perf(res.x, mu_arr, cov_arr)
            frontier_r.append(r)
            frontier_v.append(v)

else:
    # ランダムでフロンティア近似
    mu_arr, cov_arr = mu.values, cov.values
    for _ in range(10000):
        w = np.random.rand(n_assets)
        w /= w.sum()
        r, v = port_perf(w, mu_arr, cov_arr)
        frontier_r.append(r)
        frontier_v.append(v)

frontier_r = np.array(frontier_r)
frontier_v = np.array(frontier_v)

# ----------------------------------------------------------
# 6. 個別銘柄のリターン・リスク
# ----------------------------------------------------------
asset_r = mu.values
asset_v = np.sqrt(np.diag(cov.values))

# ----------------------------------------------------------
# 7. フロンティア描画
# ----------------------------------------------------------
plt.figure(figsize=(10, 7))

plt.scatter(frontier_v, frontier_r, s=15, alpha=0.4, label="Efficient Frontier")
plt.scatter(asset_v, asset_r, s=80, marker="x", color="red", label="Assets")

for i, t in enumerate(tickers):
    plt.annotate(t, (asset_v[i], asset_r[i]), fontsize=8)

plt.xlabel("Risk (Volatility, Annualized)")
plt.ylabel("Return (Annualized)")
plt.title("Efficient Frontier (Top20 Nikkei225)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


🚀 次に追加できます！

- GMV（最小分散）ポートフォリオ点をプロット
- シャープ最大ポートフォリオ（接点ポートフォリオ）
- ウェイトを棒グラフ表示
- 各銘柄の寄与度（Marginal Risk Contribution）
- PCA（イールドカーブのように株価因子分析）

どれを追加しますか？

--------------------------
## 最小分散ポートフォリオと接点ポートフォリオを求めて。リスクフリー金利を1%とし、効率的フロンティアのグラフに重ねて表示して。さらにリスクフリー金利点から接点ポートフォリオを通る接線も重ねて表示して。接線は細い青線で。

了解です！
- 最小分散ポートフォリオ(GMV)
- 接点ポートフォリオ(Tangency, rf=1%)
- 資本市場線 (CML) 

を効率的フロンティアに重ねて表示させます。

資本市場線 (CML) 
```
CML:  y = rf + ( (tan_r - rf) / tan_v ) * x
```

In [ ]:
# ==========================================================
# Nikkei225 Top20 Portfolio Optimization
# - 特殊3行ヘッダー付きCSV対応
# - 最小分散ポートフォリオ (GMV)
# - 接点ポートフォリオ (Tangency, rf = 1%)
# - 効率的フロンティア + GMV + Tangency + CML
# - アセットを赤で表示
# ==========================================================

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

# ----------------------------------------------------------
# 1. 特殊CSVを Date & Close にパースする関数
# ----------------------------------------------------------
def load_price_csv(path):
    header1 = pd.read_csv(path, nrows=1, header=None).iloc[0].tolist()
    df = pd.read_csv(path, skiprows=3, header=None)
    df.columns = header1

    df["Date"] = pd.to_datetime(df["Price"], format="%Y-%m-%d", errors="coerce")
    df["Close"] = pd.to_numeric(df["Close"], errors="coerce")

    df = df.dropna(subset=["Date", "Close"])
    out = df[["Date", "Close"]].set_index("Date")
    return out

# ----------------------------------------------------------
# 2. 20銘柄の終値を DataFrame にまとめる
# ----------------------------------------------------------
data_dir = "nikkei225_top20_prices"
prices = pd.DataFrame()

for file in sorted(os.listdir(data_dir)):
    if not file.endswith(".csv"):
        continue
    ticker = file.replace("_1y.csv", "") + ".T"
    df = load_price_csv(os.path.join(data_dir, file))
    prices[ticker] = df["Close"]

prices = prices.dropna()
display(prices.head())

# ----------------------------------------------------------
# 3. 年率リターン & 共分散
# ----------------------------------------------------------
returns = prices.pct_change().dropna()
mu = returns.mean() * 252
cov = returns.cov() * 252

tickers = prices.columns.tolist()
n_assets = len(tickers)

def port_perf(w, mu_vec, cov_mat):
    r = np.dot(w, mu_vec)
    v = np.sqrt(np.dot(w, np.dot(cov_mat, w)))
    return r, v

# ----------------------------------------------------------
# 4. 最適化
# ----------------------------------------------------------
try:
    from scipy.optimize import minimize
    use_scipy = True
except:
    use_scipy = False

mu_arr = mu.values
cov_arr = cov.values
rf = 0.01    # ★ リスクフリー金利 1%

frontier_r = []
frontier_v = []

w_gmv = w_tan = None
gmv_r = gmv_v = None
tan_r = tan_v = None

# --- SciPy の場合 ---
if use_scipy:
    w0 = np.ones(n_assets) / n_assets
    bounds = [(0,1)] * n_assets
    targets = np.linspace(mu.min(), mu.max(), 50)

    # 4-1 フロンティア（分散最小化）
    for t in targets:
        def obj(w): return np.dot(w, np.dot(cov_arr, w))
        cons = [
            {"type": "eq", "fun": lambda w: w.sum() - 1},
            {"type": "eq", "fun": lambda w, t=t: np.dot(w, mu_arr) - t},
        ]
        res = minimize(obj, w0, method="SLSQP", bounds=bounds, constraints=cons)
        if res.success:
            r, v = port_perf(res.x, mu_arr, cov_arr)
            frontier_r.append(r)
            frontier_v.append(v)

    # 4-2 GMV（最小分散）
    res_gmv = minimize(lambda w: np.dot(w, np.dot(cov_arr, w)),
                       w0, method="SLSQP",
                       bounds=bounds,
                       constraints=[{"type":"eq","fun":lambda w: w.sum()-1}])
    if res_gmv.success:
        w_gmv = res_gmv.x
        gmv_r, gmv_v = port_perf(w_gmv, mu_arr, cov_arr)

    # 4-3 接点（Sharpe最大）
    def neg_sharpe(w):
        r, v = port_perf(w, mu_arr, cov_arr)
        return -(r - rf) / v

    res_tan = minimize(neg_sharpe,
                       w0, method="SLSQP",
                       bounds=bounds,
                       constraints=[{"type":"eq","fun":lambda w: w.sum()-1}])
    if res_tan.success:
        w_tan = res_tan.x
        tan_r, tan_v = port_perf(w_tan, mu_arr, cov_arr)

# ----------------------------------------------------------
# 5. 個別銘柄プロット用
# ----------------------------------------------------------
asset_r = mu.values
asset_v = np.sqrt(np.diag(cov_arr))

# ----------------------------------------------------------
# 6. グラフ描画（CML + GMV + Tangency + Frontier）
# ----------------------------------------------------------
plt.figure(figsize=(10,7))

# 効率的フロンティア
plt.scatter(frontier_v, frontier_r, s=20, alpha=0.4, label="Efficient Frontier")

# 個別アセット
plt.scatter(asset_v, asset_r, s=70, marker="x", color="red", alpha=0.8, label="Assets")

for i, t in enumerate(tickers):
    plt.annotate(t, (asset_v[i], asset_r[i]), fontsize=8)
    

# GMV
if gmv_r is not None:
    plt.scatter(gmv_v, gmv_r, s=120, color="green", marker="o", label="GMV")
    plt.annotate("GMV", (gmv_v, gmv_r), xytext=(5,5), textcoords="offset points", color="green")

# Tangency Portfolio
if tan_r is not None:
    plt.scatter(tan_v, tan_r, s=120, color="magenta", marker="D", label="Tangency")
    plt.annotate("Tangency", (tan_v, tan_r), xytext=(5,5), textcoords="offset points", color="magenta")

    # CML（細い線）
    max_sigma = max(frontier_v) 
    sigma_grid = np.linspace(0, max_sigma, 200)
    slope = (tan_r - rf) / tan_v
    cml = rf + slope * sigma_grid

    plt.plot(sigma_grid, cml, color="blue", linewidth=0.5, label="CML (Capital Market Line)")

plt.xlabel("Risk (Volatility, Annualized)")
plt.ylabel("Return (Annualized)")
plt.title("Efficient Frontier + GMV + Tangency + CML (rf=1%)")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


グラフ上に

- 効率的フロンティア
- 個別銘柄
- 緑丸：GMV
- マゼンタ菱形：接点ポートフォリオ
- 青直線：CML（資本市場線）
    - 始点：リスクフリー金利 rf=0.01
    - 終点：接点ポートフォリオ (tan_v, tan_r)
    - 式：CML(σ) = rf + (tan_r - rf)/tan_v × σ

を描画します。

🎉 もし以下も追加したければ言ってください！

- GMV と Tangency の ウェイトを棒グラフで表示
- CML とフロンティアの 交点のプロット
- CAL（Capital Allocation Line） を描く
- シャープ比ヒートマップ
- リスク寄与度 (Marginal Risk Contribution)
- PCA（株価主成分分析）

次にどこを強化しますか？

-------
## GMV と Tangency の ウェイトを棒グラフで表示して

了解しました！

最小分散（GMV）ポートフォリオ と 接点（Tangency）ポートフォリオ の ウェイトを棒グラフで表示するコード を追加します。

✅ 追加される内容
- w_gmv（最小分散ポートフォリオのウェイト）
- w_tan（接点ポートフォリオのウェイト）
- 2つの棒グラフ
    - GMV weights
    - Tangency weights

銘柄名が横軸、ウェイトが縦軸の見やすいチャート

In [ ]:
# ==========================================================
# 7. GMV / Tangency ポートフォリオのウェイトを棒グラフで表示
# ==========================================================

import matplotlib.pyplot as plt
import numpy as np

# ----------------------------------------------------------
# GMV（最小分散ポートフォリオ）ウェイト
# ----------------------------------------------------------
if w_gmv is not None:
    plt.figure(figsize=(12,4))
    plt.bar(tickers, w_gmv, color="green", alpha=0.7)
    plt.title("GMV Portfolio Weights")
    plt.ylabel("Weight")
    plt.xticks(rotation=90)
    plt.grid(axis="y", alpha=0.3)
    plt.show()

# ----------------------------------------------------------
# Tangency（接点ポートフォリオ）ウェイト
# ----------------------------------------------------------
if w_tan is not None:
    plt.figure(figsize=(12,4))
    plt.bar(tickers, w_tan, color="magenta", alpha=0.7)
    plt.title("Tangency Portfolio Weights (Max Sharpe)")
    plt.ylabel("Weight")
    plt.xticks(rotation=90)
    plt.grid(axis="y", alpha=0.3)
    plt.show()


🌟 もし必要なら追加できます

- ウェイトの積み上げ棒グラフ（2ポートフォリオ比較）
- ウェイトの上位銘柄だけハイライト
- 数値表の出力（DataFrameでGMV/Tangencyを一覧化）
- ウェイトのヒートマップ

どれを追加しましょう？

----------
## 数値表とウェイトのヒートマップを追加して。

了解しました！
GMV（最小分散）と Tangency（接点）ポートフォリオのウェイトについて：

1. 数値表（DataFrame）で一覧表示
2. ウェイトのヒートマップを表示

この2点を追加します。

In [ ]:
# ==========================================================
# 8. GMV / Tangency のウェイト数値表を表示
# ==========================================================

import pandas as pd

weights_df = pd.DataFrame({
    "GMV": w_gmv,
    "Tangency": w_tan
}, index=tickers)

display(weights_df.style.format("{:.4f}"))

# ==========================================================
# 9. ウェイトのヒートマップ
# ==========================================================

plt.figure(figsize=(8, 6))
plt.imshow(weights_df.T, cmap="YlGnBu", aspect="auto")

plt.colorbar(label="Weight")
plt.title("Portfolio Weights Heatmap")
plt.yticks([0, 1], ["GMV", "Tangency"])
plt.xticks(range(len(tickers)), tickers, rotation=90)

plt.show()


🌟 さらに拡張できます

以下のような高度な可視化も追加可能：

- GMV & Tangency ウェイトの 積み上げ棒グラフ
- リスク寄与度（MRC / RC） のヒートマップ
- 効率的フロンティア上の複数ポートフォリオ のヒートマップ
- GMV vs Tangency の 差分ヒートマップ（どの銘柄をどれだけ増やしているか）

何を追加しますか？